In [1]:
import gc, argparse, sys, os, errno
%pylab inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#sns.set()
#sns.set_style('whitegrid')
import h5py
from PIL import Image
import os
from tqdm.notebook import tqdm
import scipy
import sklearn
from scipy.stats import pearsonr
import warnings
warnings.filterwarnings('ignore')

Populating the interactive namespace from numpy and matplotlib


In [41]:
df2 = pd.read_excel('data/指定合约.xlsx')

In [18]:
df = pd.read_excel('data/指定合约1.xlsx')
df2 = pd.read_excel('data/指定合约.xlsx')
df.head()

,year,month,指定合约,指定合约1,指定合约2
0,2021,8,TA110,TA111,TA112
1,2021,8,MA110,MA111,MA112
2,2021,8,SR111,NaN,NaN
3,2021,8,CF111,NaN,NaN
4,2021,8,RM111,NaN,NaN


In [24]:
df['指定合约1'].dropna()

0      TA111
1      MA111
7      SM111
8      SF111
9      SA111
       ...  
190    SM008
191    SF008
200    FG008
201    SM008
202    SF008
Name: 指定合约1, Length: 115, dtype: object

In [19]:
df.shape

(224, 5)

In [30]:
pd.concat([df.iloc[:, np.array([0,1,i])].dropna() for i in range(2,5)],axis=0)

,year,month,指定合约,指定合约1,指定合约2
0,2021,8,TA110,NaN,NaN
1,2021,8,MA110,NaN,NaN
2,2021,8,SR111,NaN,NaN
3,2021,8,CF111,NaN,NaN
4,2021,8,RM111,NaN,NaN
...,...,...,...,...,...
190,2020,5,NaN,NaN,SM010
191,2020,5,NaN,NaN,SF010
200,2020,4,NaN,NaN,FG010
201,2020,4,NaN,NaN,SM010


In [36]:
df.columns

Index(['year', 'month', '指定合约', '指定合约1', '指定合约2'], dtype='object')

In [61]:
df = pd.read_excel('data/指定合约1.xlsx')
df2 = pd.read_excel('data/指定合约.xlsx')
df1 = pd.concat([pd.DataFrame(np.array(df.iloc[:, np.array([0,1,i])] .dropna()),columns=['year', 'month', '指定合约'])\
               for i in range(2,5)],axis=0)

In [64]:
df1['col_all'] = df1[df1.columns ].apply(
    lambda x: ','.join(x.dropna().astype(str)),
    axis=1)
df2['col_all'] = df2[df2.columns ].apply(
    lambda x: ','.join(x.dropna().astype(str)),
    axis=1)

In [65]:
df1

,year,month,指定合约,col_all
0,2021,8,TA110,"2021,8,TA110"
1,2021,8,MA110,"2021,8,MA110"
2,2021,8,SR111,"2021,8,SR111"
3,2021,8,CF111,"2021,8,CF111"
4,2021,8,RM111,"2021,8,RM111"
...,...,...,...,...
86,2020,5,SM010,"2020,5,SM010"
87,2020,5,SF010,"2020,5,SF010"
88,2020,4,FG010,"2020,4,FG010"
89,2020,4,SM010,"2020,4,SM010"


In [66]:
df2

,year,month,合约代码,col_all
0,2021,8,TA110,"2021,8,TA110"
1,2021,8,TA111,"2021,8,TA111"
2,2021,8,TA112,"2021,8,TA112"
3,2021,8,MA110,"2021,8,MA110"
4,2021,8,MA111,"2021,8,MA111"
...,...,...,...,...
448,2020,1,MA003,"2020,1,MA003"
449,2020,1,SR003,"2020,1,SR003"
450,2020,1,CF003,"2020,1,CF003"
451,2020,1,RM007,"2020,1,RM007"


In [67]:
np.setdiff1d(df2['col_all'],df1['col_all'])

array(['2021,7,CY201'], dtype=object)

In [76]:
np.unique(df2['col_all'],return_counts=1)[0][np.unique(df2['col_all'],return_counts=1)[1]>1]

array(['2021,7,CF111', '2021,7,MA110', '2021,7,MA111', '2021,7,MA112',
       '2021,7,OI111', '2021,7,PF110', '2021,7,PF111', '2021,7,PF112',
       '2021,7,RM111', '2021,7,SA110', '2021,7,SA111', '2021,7,SA112',
       '2021,7,SF110', '2021,7,SF111', '2021,7,SF112', '2021,7,SM110',
       '2021,7,SM111', '2021,7,SM112', '2021,7,SR111', '2021,7,TA110',
       '2021,7,TA111', '2021,7,TA112'], dtype=object)

In [79]:
df2.iloc[30:100]

,year,month,合约代码,col_all
30,2021,7,CF111,"2021,7,CF111"
31,2021,7,RM111,"2021,7,RM111"
32,2021,7,CY201,"2021,7,CY201"
33,2021,7,OI111,"2021,7,OI111"
34,2021,7,SM110,"2021,7,SM110"
...,...,...,...,...
95,2021,6,SA111,"2021,6,SA111"
96,2021,6,PF110,"2021,6,PF110"
97,2021,6,PF111,"2021,6,PF111"
98,2021,5,TA107,"2021,5,TA107"


In [69]:
df2.drop_duplicates

<bound method DataFrame.drop_duplicates of      year  month   合约代码       col_all
0    2021      8  TA110  2021,8,TA110
1    2021      8  TA111  2021,8,TA111
2    2021      8  TA112  2021,8,TA112
3    2021      8  MA110  2021,8,MA110
4    2021      8  MA111  2021,8,MA111
..    ...    ...    ...           ...
448  2020      1  MA003  2020,1,MA003
449  2020      1  SR003  2020,1,SR003
450  2020      1  CF003  2020,1,CF003
451  2020      1  RM007  2020,1,RM007
452  2020      1  CY005  2020,1,CY005

[453 rows x 4 columns]>